# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [66]:
# Importing the dataset
pokemon = pd.read_csv('pokemon.csv', sep =",")
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [67]:
# your code here

#Hypothesis: Total=(HP+ Attack+Defense+Sp. Atk+ Sp. Def+ Speed).
pokemon['Estimation'] = pokemon['HP']+pokemon['Attack']+pokemon['Defense']+pokemon['Sp. Atk']+pokemon['Sp. Def']+pokemon['Speed']-pokemon['Total']

pokemon.head(10)

#pokemon['Estimation']
pokemon_ordered = pokemon.sort_values(by='Estimation', ascending=False) #los ordeno de mayor a menor
pokemon_ordered.head(10) 

#Hypothesis correct. It works for all pokemons!

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Estimation
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,0
537,480,Uxie,Psychic,NaN,580,75,75,130,75,130,95,4,True,0
527,475,GalladeMega Gallade,Psychic,Fighting,618,68,165,95,65,115,110,4,False,0
528,476,Probopass,Rock,Steel,525,60,55,145,75,150,40,4,False,0
529,477,Dusknoir,Ghost,NaN,525,45,100,135,65,135,45,4,False,0
530,478,Froslass,Ice,Ghost,480,70,80,70,80,70,110,4,False,0
531,479,Rotom,Electric,Ghost,440,50,50,77,95,77,91,4,False,0
532,479,RotomHeat Rotom,Electric,Fire,520,50,65,107,105,107,86,4,False,0
533,479,RotomWash Rotom,Electric,Water,520,50,65,107,105,107,86,4,False,0
534,479,RotomFrost Rotom,Electric,Ice,520,50,65,107,105,107,86,4,False,0


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [68]:
# your code here
##One Hot Encoding approach: 

pokemon_dum = pd.get_dummies(pokemon, columns = ['Type 1','Type 2'])
pokemon_dum = pokemon_dum.rename(columns={'Type 1_Grass': 'Grass', 'Type 1_Poison':'Poison','Type 1_Fire':'Fire','Type 1_Flying':'Flying','Type 1_Dragon':'Dragon','Type 1_Water':'Water','Type 1_Bug':'Bug','Type 1_Normal':'Normal','Type 1_Electric':'Electric','Type 1_Ground':'Ground','Type 1_Fairy':'Fairy','Type 1_Fighting':'Fighting','Type 1_Psychic':'Psychic','Type 1_Rock':'Rock','Type 1_Steel':'Steel','Type 1_Ice':'Ice','Type 1_Ghost':'Ghost','Type 1_Dark':'Dark'})
pokemon_dum.head()


,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,1,Bulbasaur,318,45,49,49,65,65,45,1,...,0,0,0,0,0,1,0,0,0,0
1,2,Ivysaur,405,60,62,63,80,80,60,1,...,0,0,0,0,0,1,0,0,0,0
2,3,Venusaur,525,80,82,83,100,100,80,1,...,0,0,0,0,0,1,0,0,0,0
3,3,VenusaurMega Venusaur,625,80,100,123,122,120,80,1,...,0,0,0,0,0,1,0,0,0,0
4,4,Charmander,309,39,52,43,60,50,65,1,...,0,0,0,0,0,0,0,0,0,0


In [69]:
#Add values
pokemon_dum[pokemon_dum['Grass']==0]['Grass'] = pokemon_dum['Grass'].add(pokemon_dum['Type 2_Grass'])
pokemon_dum[pokemon_dum['Poison']==0]['Poison'] = pokemon_dum['Poison'].add(pokemon_dum['Type 2_Poison'])
pokemon_dum[pokemon_dum['Fire']==0]['Fire'] = pokemon_dum['Fire'].add(pokemon_dum['Type 2_Fire'])
pokemon_dum[pokemon_dum['Flying']==0]['Flying'] = pokemon_dum['Flying'].add(pokemon_dum['Type 2_Flying'])
pokemon_dum[pokemon_dum['Dragon']==0]['Dragon'] = pokemon_dum['Dragon'].add(pokemon_dum['Type 2_Dragon'])
pokemon_dum[pokemon_dum['Water']==0]['Water'] = pokemon_dum['Water'].add(pokemon_dum['Type 2_Water'])
pokemon_dum[pokemon_dum['Bug']==0]['Bug'] = pokemon_dum['Bug'].add(pokemon_dum['Type 2_Bug'])
pokemon_dum[pokemon_dum['Normal']==0]['Normal'] = pokemon_dum['Normal'].add(pokemon_dum['Type 2_Normal'])
pokemon_dum[pokemon_dum['Electric']==0]['Electric'] = pokemon_dum['Electric'].add(pokemon_dum['Type 2_Electric'])
pokemon_dum[pokemon_dum['Ground']==0]['Ground'] = pokemon_dum['Ground'].add(pokemon_dum['Type 2_Ground'])
pokemon_dum[pokemon_dum['Fairy']==0]['Fairy'] = pokemon_dum['Fairy'].add(pokemon_dum['Type 2_Fairy'])
pokemon_dum[pokemon_dum['Fighting']==0]['Fighting'] = pokemon_dum['Fighting'].add(pokemon_dum['Type 2_Fighting'])
pokemon_dum[pokemon_dum['Psychic']==0]['Psychic'] = pokemon_dum['Psychic'].add(pokemon_dum['Type 2_Psychic'])
pokemon_dum[pokemon_dum['Rock']==0]['Rock'] = pokemon_dum['Rock'].add(pokemon_dum['Type 2_Rock'])
pokemon_dum[pokemon_dum['Steel']==0]['Steel'] = pokemon_dum['Steel'].add(pokemon_dum['Type 2_Steel'])
pokemon_dum[pokemon_dum['Ice']==0]['Ice'] = pokemon_dum['Ice'].add(pokemon_dum['Type 2_Ice'])
pokemon_dum[pokemon_dum['Ghost']==0]['Ghost'] = pokemon_dum['Ghost'].add(pokemon_dum['Type 2_Ghost'])
pokemon_dum[pokemon_dum['Dark']==0]['Dark'] = pokemon_dum['Dark'].add(pokemon_dum['Type 2_Dark'])
pokemon_dum.columns



C:\Users\rozio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\rozio\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\rozio\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

Index(['#', 'Name', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def',
       'Speed', 'Generation', 'Legendary', 'Estimation', 'Bug', 'Dark',
       'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost',
       'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock',
       'Steel', 'Water', 'Type 2_Bug', 'Type 2_Dark', 'Type 2_Dragon',
       'Type 2_Electric', 'Type 2_Fairy', 'Type 2_Fighting', 'Type 2_Fire',
       'Type 2_Flying', 'Type 2_Ghost', 'Type 2_Grass', 'Type 2_Ground',
       'Type 2_Ice', 'Type 2_Normal', 'Type 2_Poison', 'Type 2_Psychic',
       'Type 2_Rock', 'Type 2_Steel', 'Type 2_Water'],
      dtype='object')

In [71]:
#Drop columns that are not required
pokemon_dum = pokemon_dum.drop('Type 2_Grass', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Poison', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Fire', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Flying', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Dragon', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Water', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Bug', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Normal', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Electric', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Ground', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Fairy', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Fighting', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Psychic', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Rock', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Steel', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Ice', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Ghost', axis=1)
pokemon_dum = pokemon_dum.drop('Type 2_Dark', axis=1)
pokemon_dum.columns


KeyError: "['Type 2_Grass'] not found in axis"

In [72]:
pokemon_dum.head(10)

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,1,Bulbasaur,318,45,49,49,65,65,45,1,...,0,1,0,0,0,0,0,0,0,0
1,2,Ivysaur,405,60,62,63,80,80,60,1,...,0,1,0,0,0,0,0,0,0,0
2,3,Venusaur,525,80,82,83,100,100,80,1,...,0,1,0,0,0,0,0,0,0,0
3,3,VenusaurMega Venusaur,625,80,100,123,122,120,80,1,...,0,1,0,0,0,0,0,0,0,0
4,4,Charmander,309,39,52,43,60,50,65,1,...,0,0,0,0,0,0,0,0,0,0
5,5,Charmeleon,405,58,64,58,80,65,80,1,...,0,0,0,0,0,0,0,0,0,0
6,6,Charizard,534,78,84,78,109,85,100,1,...,0,0,0,0,0,0,0,0,0,0
7,6,CharizardMega Charizard X,634,78,130,111,130,85,100,1,...,0,0,0,0,0,0,0,0,0,0
8,6,CharizardMega Charizard Y,634,78,104,78,159,115,100,1,...,0,0,0,0,0,0,0,0,0,0
9,7,Squirtle,314,44,48,65,50,64,43,1,...,0,0,0,0,0,0,0,0,0,1


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [84]:
# your code here
dict_correl = {}
dummies = 'Bug', 'Dark','Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost','Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock','Steel', 'Water'
for element in dummies:
        clave = element
        valor = pokemon_dum['Total'].corr(pokemon_dum[element])
        dict_correl[clave] = valor
dict_correl

{'Bug': -0.14395700406987763,
 'Dark': 0.017818059481941925,
 'Dragon': 0.196531656074697,
 'Electric': 0.01671523185109553,
 'Fairy': -0.02694807292749806,
 'Fighting': -0.029085910202663833,
 'Fire': 0.050526548143285874,
 'Flying': 0.02950369369685115,
 'Ghost': 0.007593699022931042,
 'Grass': -0.03605670522272074,
 'Ground': 0.0040820388806002954,
 'Ice': -0.0024118191624834828,
 'Normal': -0.10415013081335948,
 'Poison': -0.05712285130498482,
 'Psychic': 0.0943636354793588,
 'Rock': 0.03752409253743022,
 'Steel': 0.08199964261669279,
 'Water': -0.01563957537388821}

In [91]:
valor_maximo = max(dict_correl.values())
valor_maximo
dict_correl.keys

claves = [clave for clave, valor in dict_correl.items() if valor == valor_maximo]
claves
print("The one that as more correlationship with the Total metric is",claves)

The one that as more correlationship with the Total metric is ['Dragon']


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [113]:
# your code here
valor_maximo = max(dict_correl.values())
valor_maximo
dict_correl.keys

claves = [clave for clave, valor in dict_correl.items() if valor == valor_maximo]
claves
print("The one that as more correlationship with the Total metric is",claves, "with a correlationship of",valor_maximo)

#Another approach:

groupby_type1 = pokemon.groupby(['Type 1']).agg({'Total':'mean'}).reset_index().head()
groupby_type2 =pokemon.groupby(['Type 2']).agg({'Total':'mean'}).reset_index().head()

print(groupby_type1)
print("------------------------")
print(groupby_type2)
##The best option is to choose a fairy pokemon.

print("The best option is to choose a pokemon with type 1 or 2 is Dragon due to the maximum value of the mean:")

print(groupby_type1[groupby_type1['Total']==groupby_type1['Total'].max()])
print(groupby_type2[groupby_type2['Total']==groupby_type2['Total'].max()])

The one that as more correlationship with the Total metric is ['Dragon'] with a correlationship of 0.196531656074697
     Type 1       Total
0       Bug  378.927536
1      Dark  445.741935
2    Dragon  550.531250
3  Electric  443.409091
4     Fairy  413.176471
------------------------
     Type 2       Total
0       Bug  393.333333
1      Dark  484.400000
2    Dragon  526.166667
3  Electric  455.333333
4     Fairy  417.956522
The best option is to choose a pokemon with type 1 or 2 is Dragon due to the maximum value of the mean:
   Type 1      Total
2  Dragon  550.53125
   Type 2       Total
2  Dragon  526.166667
